In [12]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.model_selection import train_test_split

In [13]:
combined_stocks_df = pd.read_csv("filtered_stocks_combined.csv")

In [14]:
combined_stocks_df

,Date,index,Open,High,Low,Close,Volume,OpenInt,ticker
0,2011-01-03,45725,41.600,42.293,41.600,42.204,123930383,0,AAPL
1,2011-01-04,45726,42.579,42.579,42.023,42.426,86135637,0,AAPL
2,2011-01-05,45727,42.260,42.817,42.196,42.772,70669988,0,AAPL
3,2011-01-06,45728,42.902,42.932,42.632,42.739,83619699,0,AAPL
4,2011-01-07,45729,42.787,43.073,42.503,43.045,86506108,0,AAPL
...,...,...,...,...,...,...,...,...,...
75495,2016-12-23,14745947,87.556,87.625,87.201,87.383,4428429,0,XOM
75496,2016-12-27,14745948,87.499,87.768,87.257,87.423,5100402,0,XOM
75497,2016-12-28,14745949,87.354,87.768,86.949,86.989,6834213,0,XOM
75498,2016-12-29,14745950,86.797,87.277,86.728,87.036,6938299,0,XOM


In [15]:
# Convert Date to datetime and set as index
combined_stocks_df["Date"] = pd.to_datetime(combined_stocks_df["Date"])
combined_stocks_df.set_index("Date", inplace=True)

# Drop unnecessary columns if any
combined_stocks_df = combined_stocks_df.drop(columns=["index"])  # Optional

# Pivot to multi-level columns: Ticker as level 1, feature as level 2
stocks_df = combined_stocks_df.pivot_table(
    index=combined_stocks_df.index,
    columns="ticker",
    values=[col for col in combined_stocks_df.columns if col != "ticker"]
)

# Sort columns for clarity
stocks_df = stocks_df.sort_index(axis=1, level=0)


# Swap the column MultiIndex levels
stocks_df_leveled = stocks_df.swaplevel(axis=1)

# Sort by ticker (Level 0)
stocks_df_leveled = stocks_df_leveled.sort_index(axis=1, level=0)

# Preview the new structure
stocks_df_leveled.head()


ticker        AAPL                                                  ABT  \
             Close    High     Low    Open OpenInt       Volume   Close   
Date                                                                      
2011-01-03  42.204  42.293  41.600  41.600     0.0  123930383.0  19.358   
2011-01-04  42.426  42.579  42.023  42.579     0.0   86135637.0  19.540   
2011-01-05  42.772  42.817  42.196  42.260     0.0   70669988.0  19.540   
2011-01-06  42.739  42.932  42.632  42.902     0.0   83619699.0  19.500   
2011-01-07  43.045  43.073  42.503  42.787     0.0   86506108.0  19.582   

ticker                              ...     WMT                              \
              High     Low    Open  ...     Low    Open OpenInt      Volume   
Date                                ...                                       
2011-01-03  19.549  19.338  19.549  ...  46.027  46.089     0.0  16789275.0   
2011-01-04  19.561  19.350  19.448  ...  46.139  46.419     0.0  14296931.0   
2011-01-05  19.764  19.460  19.520  ...  46.174  46.539     0.0  16723328.0   
2011-01-06  19.682  19.342  19.634  ...  45.712  46.166     0.0  18335156.0   
2011-01-07  19.615  19.444  19.489  ...  45.698  45.810     0.0   9374462.0   

ticker         XOM                                              
             Close    High     Low    Open OpenInt      Volume  
Date                                                            
2011-01-03  60.341  60.535  59.604  59.668     0.0  28807445.0  
2011-01-04  60.625  60.665  60.235  60.469     0.0  24744869.0  
2011-01-05  60.462  60.600  60.009  60.430     0.0  20448359.0  
2011-01-06  60.853  61.052  60.439  60.625     0.0  27829692.0  
2011-01-07  61.182  61.431  60.777  60.876     0.0  23838996.0  

[5 rows x 300 columns]

In [16]:
# 3. Time-based split into 60% train, 20% val, 20% test
train_dict, val_dict, test_dict = {}, {}, {}

for ticker in stocks_df_leveled.columns.levels[0]:
    stocks_df_leveled.loc[:, (ticker, 'log_return')] = np.log(
    stocks_df_leveled[ticker]['Close'] / stocks_df_leveled[ticker]['Close'].shift(1)
)

    df = stocks_df_leveled[ticker].dropna().sort_index()
    total_len = len(df)
    train_end = int(total_len * 0.6)
    val_end = train_end + int(total_len * 0.2)

    train_dict[ticker] = df.iloc[:train_end]
    val_dict[ticker] = df.iloc[train_end:val_end]
    test_dict[ticker] = df.iloc[val_end:]

train_df = pd.concat(train_dict, names=["Ticker", "Date"])
val_df = pd.concat(val_dict, names=["Ticker", "Date"])
test_df = pd.concat(test_dict, names=["Ticker", "Date"])


In [17]:
train_df

Close    High     Low    Open  OpenInt       Volume  \
Ticker Date                                                               
AAPL   2011-01-04  42.426  42.579  42.023  42.579      0.0   86135637.0   
       2011-01-05  42.772  42.817  42.196  42.260      0.0   70669988.0   
       2011-01-06  42.739  42.932  42.632  42.902      0.0   83619699.0   
       2011-01-07  43.045  43.073  42.503  42.787      0.0   86506108.0   
       2011-01-10  43.855  43.956  43.179  43.393      0.0  124888228.0   
...                   ...     ...     ...     ...      ...          ...   
XOM    2014-08-04  88.145  88.411  86.542  86.921      0.0   13804459.0   
       2014-08-05  86.445  87.625  86.077  87.492      0.0   14847864.0   
       2014-08-06  87.133  87.723  86.445  86.445      0.0   11264688.0   
       2014-08-07  86.507  87.889  85.944  87.828      0.0   11379351.0   
       2014-08-08  87.800  87.847  86.551  86.947      0.0   12730751.0   

                   log_return  
Ticker Date                    
AAPL   2011-01-04    0.005246  
       2011-01-05    0.008122  
       2011-01-06   -0.000772  
       2011-01-07    0.007134  
       2011-01-10    0.018643  
...                       ...  
XOM    2014-08-04    0.013374  
       2014-08-05   -0.019475  
       2014-08-06    0.007927  
       2014-08-07   -0.007210  
       2014-08-08    0.014836  

[45250 rows x 7 columns]

In [20]:
# Function to create rolling window sequences
def create_rolling_window_data(df, window_size=10, target_shift=1):
    X, y = [], []
    tickers = df.columns.levels[0]  # Level 0 is tickers after swaplevel
    num_tickers = len(tickers)
    
    for i in range(window_size, len(df) - target_shift):
        window = df.iloc[i - window_size:i]
        X.append(window.values.flatten())
        
        next_day = df.iloc[i + target_shift - 1]
        targets = []
        for ticker in tickers:
            targets.append(next_day[(ticker, 'Open')])
            targets.append(next_day[(ticker, 'Close')])
        y.append(targets)
    
    return np.array(X), np.array(y), num_tickers

# Create sequences from the preprocessed DataFrame
X, y, num_tickers = create_rolling_window_data(stocks_df_leveled)

# Split data into train, validation, and test sets (60%, 20%, 20%)
total_size = len(X)
train_size = int(0.6 * total_size)
val_size = int(0.2 * total_size)
test_size = total_size - train_size - val_size

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size + val_size], y[train_size:train_size + val_size]
X_test, y_test = X[train_size + val_size:], y[train_size + val_size:]

# Normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Build the MLP model
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(256, activation='relu'),
    Dense(num_tickers * 2)  # Output layer: number of tickers * 2 (open and close)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=100,
    batch_size=32,
    verbose=1
)

# Evaluate the model
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Test Mean Squared Error (MSE): {mse:.4f}")
print(f"Test Mean Absolute Error (MAE): {mae:.4f}")

# Calculate directional accuracy for close prices
actual_close = y_test[:, 1::2]  # Close prices (every second element)
predicted_close = y_pred[:, 1::2]
features = ['Open', 'Close', 'High', 'Low', 'Volume']
close_idx = features.index('Close')  # 1
num_features = len(features)  # 5
window_size = 10
positions = [(window_size - 1) * num_tickers * num_features + i * num_features + close_idx for i in range(num_tickers)]
last_close = X_test[:, positions]

actual_change = actual_close - last_close
predicted_change = predicted_close - last_close
directional_accuracy = np.mean(np.sign(actual_change) == np.sign(predicted_change))

print(f"Directional Accuracy for Close Prices: {directional_accuracy:.4f}")

Epoch 1/100


c:\Users\gao23\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 7971.8940 - val_loss: 16212.7705
Epoch 2/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 8118.3633 - val_loss: 16183.0166
Epoch 3/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 7890.3804 - val_loss: 16124.9229
Epoch 4/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 7971.2441 - val_loss: 16017.2490
Epoch 5/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 7909.8320 - val_loss: 15840.0723
Epoch 6/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 7580.3950 - val_loss: 15581.1104
Epoch 7/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 7383.7891 - val_loss: 15234.1221
Epoch 8/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 7265.6162 - val_loss: 14802.7578
Epoch 9/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6976.6162 - val_loss: 14299.6924
Epoch 10/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6724.1636 - val_loss: 13740.1191
Epoch 11/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 6166.3818 - val_loss

In [21]:
# Function to create rolling window sequences (unchanged)
def create_rolling_window_data(df, window_size=10, target_shift=1):
    X, y = [], []
    tickers = df.columns.levels[0]
    num_tickers = len(tickers)
    
    for i in range(window_size, len(df) - target_shift):
        window = df.iloc[i - window_size:i]
        X.append(window.values.flatten())
        next_day = df.iloc[i + target_shift - 1]
        targets = []
        for ticker in tickers:
            targets.append(next_day[(ticker, 'Open')])
            targets.append(next_day[(ticker, 'Close')])
        y.append(targets)
    
    return np.array(X), np.array(y), num_tickers

# Function to train and evaluate a model for a given window size
def evaluate_window_size(df, window_size):
    X, y, num_tickers = create_rolling_window_data(df, window_size=window_size)
    
    # Split data (60% train, 20% val, 20% test)
    total_size = len(X)
    train_size = int(0.6 * total_size)
    val_size = int(0.2 * total_size)
    test_size = total_size - train_size - val_size
    
    X_train, y_train = X[:train_size], y[:train_size]
    X_val, y_val = X[train_size:train_size + val_size], y[train_size:train_size + val_size]
    X_test, y_test = X[train_size + val_size:], y[train_size + val_size:]
    
    # Normalize
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    # Build and train model
    model = Sequential([
        Dense(512, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        Dense(256, activation='relu'),
        Dense(num_tickers * 2)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val), 
              epochs=50, batch_size=32, verbose=0)  # Silent training
    
    # Evaluate on validation set
    y_val_pred = model.predict(X_val_scaled, verbose=0)
    val_mpe = np.mean((y_val - y_val_pred) / (y_val + 1e-10)) * 100
    return val_mpe, X, y, num_tickers

# Test window sizes and find the best
window_sizes = range(5, 21)  # Test 5 to 20 days
mpe_scores = {}
for window_size in window_sizes:
    val_mpe, X, y, num_tickers = evaluate_window_size(stocks_df_leveled, window_size)
    mpe_scores[window_size] = val_mpe
    print(f"Window Size {window_size}: Validation MPE = {val_mpe:.4f}%")

best_window_size = min(mpe_scores, key=mpe_scores.get)  # Lowest MPE (least negative or smallest positive)
print(f"Best Window Size: {best_window_size} with Validation MPE: {mpe_scores[best_window_size]:.4f}%")

# Final model with best window size
X, y, num_tickers = create_rolling_window_data(stocks_df_leveled, window_size=best_window_size)
total_size = len(X)
train_size = int(0.6 * total_size)
val_size = int(0.2 * total_size)
test_size = total_size - train_size - val_size

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size + val_size], y[train_size:train_size + val_size]
X_test, y_test = X[train_size + val_size:], y[train_size + val_size:]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(256, activation='relu'),
    Dense(num_tickers * 2)
])
model.compile(optimizer='adam', loss='mse')
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=50,
    batch_size=32,
    verbose=1
)

# Evaluate with MPE
y_pred = model.predict(X_test_scaled)
mpe = np.mean((y_test - y_pred) / (y_test + 1e-10)) * 100
print(f"Test Mean Percentage Error (MPE): {mpe:.4f}%")

# Directional accuracy
actual_close = y_test[:, 1::2]
predicted_close = y_pred[:, 1::2]
features = ['Open', 'Close', 'High', 'Low', 'Volume']
close_idx = features.index('Close')
num_features = len(features)
positions = [(best_window_size - 1) * num_tickers * num_features + i * num_features + close_idx for i in range(num_tickers)]
last_close = X_test[:, positions]
actual_change = actual_close - last_close
predicted_change = predicted_close - last_close
directional_accuracy = np.mean(np.sign(actual_change) == np.sign(predicted_change))
print(f"Directional Accuracy for Close Prices: {directional_accuracy:.4f}")

c:\Users\gao23\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Window Size 5: Validation MPE = 24.3605%


c:\Users\gao23\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Window Size 6: Validation MPE = 25.4227%


c:\Users\gao23\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

Window Size 19: Validation MPE = 24.7027%


c:\Users\gao23\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Window Size 20: Validation MPE = 24.6962%
Best Window Size: 5 with Validation MPE: 24.3605%
Epoch 1/50


c:\Users\gao23\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 6173.6865 - val_loss: 16180.3271
Epoch 2/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 7760.9883 - val_loss: 16158.6445
Epoch 3/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 7966.4126 - val_loss: 16135.2588
Epoch 4/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 7821.6406 - val_loss: 16108.7266
Epoch 5/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 7789.2051 - val_loss: 16076.6318
Epoch 6/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 7881.1211 - val_loss: 16036.3887
Epoch 7/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 7868.3691 - val_loss: 15985.8799
Epoch 8/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 7704.4951 - val_loss: 15922.5547
Epoch 9/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 7707.7935 - val_loss: 15843.2852
Epoch 10/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 7730.7793 - val_loss: 15745.3301
Epoch 11/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 7492.2769 - val_loss: 15626.2432
Epoch 1